In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
#from resnets_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [2]:
import os
import numpy as np
import tensorflow as tf
import h5py
import math

def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters
    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3'] 
                                                           # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3
    
    return Z3

def predict(X, parameters):
    
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
img_height,img_width = 64,64 
num_classes = 10
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [15]:

import numpy as np
import pandas as pd
#from models import VGG16Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Data loading
X_train = pd.read_pickle('/content/drive/My Drive/train.pkl')[..., np.newaxis]
y_train = pd.read_csv('/content/drive/My Drive/train_y.csv', index_col=0)
X_test = pd.read_pickle('/content/drive/My Drive/test.pkl')[..., np.newaxis]

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,  # mudei aqui antes era 30
    shear_range=0.2,
    zoom_range=0.2)
    #featurewise_center=True) # acc val lixo 
    #featurewise_std_normalization=True) #acc val lixo
    #zca_whitening=True)#, !!!! esse parametro sobrepoe o featurewise_std_normalization
    #width_shift_range=0.2, 
    #height_shift_range=0.2,
    #horizontal_flip=True, 
    #vertical_flip=True) 
'''
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen = ImageDataGenerator(zca_whitening=True)
datagen = ImageDataGenerator(rotation_range=90)

shift = 0.2
datagen = ImageDataGenerator(width_shift_range=shift, height_shift_range=shift)

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)'''




datagen.fit(X_train)
datagen.fit(X_test)

# One-hot encoding
y_train = to_categorical(y_train)

# Training and validation split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


from keras.optimizers import SGD, Adam
#sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy']) #estava usando esse o ADAM
#model.compile(optimizer=SGD(learning_rate=0.005, momentum=0.8, decay=0.00003125), loss='categorical_crossentropy', metrics=['accuracy'])#agora vous usar SG Vincent


# Model training
#model = VGG16Model()
callback = EarlyStopping(monitor='val_accuracy', patience=50, verbose=1)
model.fit(datagen.flow(X_train, y_train),batch_size=64, validation_data=(X_valid, y_valid), epochs=100, callbacks=[callback], verbose=2)





Epoch 1/100
1000/1000 - 45s - loss: 0.1094 - accuracy: 0.9669 - val_loss: 0.4750 - val_accuracy: 0.8991
Epoch 2/100
1000/1000 - 44s - loss: 0.1099 - accuracy: 0.9664 - val_loss: 0.4348 - val_accuracy: 0.9065
Epoch 3/100
1000/1000 - 44s - loss: 0.1858 - accuracy: 0.9453 - val_loss: 2.2090 - val_accuracy: 0.7735
Epoch 4/100
1000/1000 - 44s - loss: 0.2459 - accuracy: 0.9319 - val_loss: 0.4731 - val_accuracy: 0.8831
Epoch 5/100
1000/1000 - 44s - loss: 0.1228 - accuracy: 0.9595 - val_loss: 0.4469 - val_accuracy: 0.8946
Epoch 6/100
1000/1000 - 44s - loss: 0.1088 - accuracy: 0.9667 - val_loss: 0.5189 - val_accuracy: 0.8896
Epoch 7/100
1000/1000 - 44s - loss: 0.0974 - accuracy: 0.9696 - val_loss: 0.4900 - val_accuracy: 0.8951
Epoch 8/100
1000/1000 - 44s - loss: 0.1039 - accuracy: 0.9673 - val_loss: 0.4630 - val_accuracy: 0.8939
Epoch 9/100
1000/1000 - 44s - loss: 0.1059 - accuracy: 0.9669 - val_loss: 0.4595 - val_accuracy: 0.9001
Epoch 10/100
1000/1000 - 44s - loss: 0.1084 - accuracy: 0.9658 -

In [16]:
# Model testing
predictions = pd.DataFrame(np.argmax(model.predict(X_test), axis=1), columns=['label'])
predictions.index.name = 'id'
predictions.to_csv('/content/drive/My Drive/results_resnet_keras_original_bs_externo.csv')